In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Transformation that converts the image to tensor and flattens it
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert image to tensor (1, 28, 28)
    transforms.Lambda(lambda x: x.flatten())  # Flatten the tensor (28*28 = 784)
])

# Load the MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Create DataLoader for training and testing
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the network architecture
class SimpleFCNet(nn.Module):
    def __init__(self):
        super(SimpleFCNet, self).__init__()
        # Input is flattened (784), output is 10 classes (digits 0-9)
        self.fc_layers = nn.Sequential(
            nn.Linear(28 * 28, 32),  # First fully connected layer: 784 -> 32
            nn.ReLU(),               # ReLU activation
            nn.Linear(32, 16),       # Second fully connected layer: 32 -> 16
            nn.ReLU(),               # ReLU activation
            nn.Linear(16, 10)        # Output layer: 16 -> 10 (number of classes)
        )

    def forward(self, x):
        return self.fc_layers(x)

# Initialize the neural network, loss function, and optimizer
model = SimpleFCNet()
criterion = nn.CrossEntropyLoss()  # Loss function for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass: compute predictions and loss
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate the loss
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Evaluate the model on the test set
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient calculation for evaluation
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")

# ---- Export to ONNX ----
dummy_input = torch.randn(1, 28 * 28)  # Dummy input (flattened size 784)
onnx_file_path = "mnist_fc_model.onnx"
torch.onnx.export(model, dummy_input, onnx_file_path, input_names=["input"], output_names=["output"], opset_version=11)
print(f"Model exported to {onnx_file_path}")


Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

Epoch [1/5], Loss: 0.5453
Epoch [2/5], Loss: 0.2452
Epoch [3/5], Loss: 0.1933
Epoch [4/5], Loss: 0.1643
Epoch [5/5], Loss: 0.1462
Accuracy on test set: 95.42%
============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Model exported to mnist_fc_model.onnx
